# Final Project for Applied Physics in Programming
By: Eveliina Hampus, DIN22SP

1. Import necessary libraries. 

Remember to install libraries with %pip install or properly via CLI with pip install.
Notice that Oceanborn is depentant from Matplotlib.

In [5]:
import numpy as np # For calculations
import matplotlib.pyplot as plt # For basic plotting
import pandas as pd # For data manipulation
import sympy as sm # For symbolic calculations
import scipy as sc # For scientific processing
import matplotlib as mpl # To support oceanborn libary
import seaborn as sns # For more informative plots, requires matplotlib to be imported first
from scipy.signal import butter,filtfilt # For filtering
from math import radians, cos, sin, asin, sqrt #
from sympy.abc import x #

2. Read the data using pandas dataframes.

You may to explore your data with few methods to get basic understanding your data. Use these with a variable assigned to your dataframe.

.head() prints out first 5 lines of the data, whilst .tail() does same for last 5 lines.
.info() prints out information of datatypes in the data.
.describe() shows basic statistics of the data.

In [9]:
gps_data = pd.read_csv('GPS_data.csv') # Read the GPS data from the csv file
acc_data = pd.read_csv('Acceleration_data.csv') # Read the acceleration data from the csv file

gps_data.head() # Show the first 5 rows of the data
acc_data.head()

,Unnamed: 0,seconds_elapsed,z,y,x
0,0,0.143440,-2.846008,-0.581977,-0.468246
1,1,0.159217,-2.651466,-0.560432,-0.418557
2,2,0.175117,-2.132024,-0.366186,-0.535990
3,3,0.190986,-1.023350,-0.041039,-0.522485
4,4,0.206763,-0.449766,0.229015,-0.356920


In [10]:
gps_data.describe() # Show the basic statistics of the data
acc_data.describe()

,Unnamed: 0,seconds_elapsed,z,y,x
count,8446.00000,8446.000000,8446.000000,8446.000000,8446.000000
mean,4222.50000,67.052698,0.238562,-0.547187,-0.371282
std,2438.29452,38.640540,3.771963,1.876145,1.471625
min,0.00000,0.143440,-9.976121,-6.470728,-6.553377
25%,2111.25000,33.591620,-2.434875,-1.858556,-1.198304
50%,4222.50000,67.052189,-0.047324,-0.573834,-0.296514
75%,6333.75000,100.507907,2.700927,0.616766,0.490000
max,8445.00000,133.971151,15.250758,6.671381,6.245217
